In [1]:
import findspark
findspark.init()

In [15]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import (
    col, year, round, rand
)

import os

In [3]:
spark = (
    SparkSession
    .builder
    .appName('Partitioning I')
    .getOrCreate()
)

In [11]:
base_path = os.getcwd()

project_path = ('/').join(base_path.split('/')[0:-2]) 

questions_input_path = os.path.join(project_path, 'output/questions-transformed')

output_path_I = os.path.join(project_path, 'output/1/questions-partitioned')
output_path_II = os.path.join(project_path, 'output/2/questions-partitioned')


# Task I

* partition questions by year (derived from creation_date) and make one file per folder

In [8]:
questionsDF = (
    spark
    .read
    .option('path', questions_input_path)
    .load()
    .withColumn('year', year('creation_date'))
)

In [10]:
(
    questionsDF
    .repartition('year')
    .write
    .mode('overwrite')
    .partitionBy('year')
    .option('path', output_path_I)
    .save()
)

# Task II

* partition questions by year (derived from creation_date) and make five files per folder

In [16]:
(
    questionsDF
    .repartition('year', round(round(rand(12), 2) * 100) % 5)
    .write
    .mode('overwrite')
    .partitionBy('year')
    .option('path', output_path_II)
    .save()
)

In [17]:
spark.stop()